# OTA Camera (Daylight Settings)

This notebook is about exploring the ASICamera2 API for use with the ASI482MC as a camera through the Optical Tube Assembly (OTA) for remote control and exploration of the night sky.

## Identify Camera by Number

In [ ]:
import asi

cameras = asi.getConnectedCameras()
print("Connected camera's:")

for camera_number in cameras:
    camera_name = (cameras[camera_number]['Name']).decode()
    print(f'  Camera Number: {camera_number} ({camera_name})')


### Playing with exposure and gain settings based on use (Finder Camera).
Natural daylight: exposure around 300,000 us. gain is 0

Night observing (decent image feedback): exposure around 2,000,000 us. gain around 80

Night observing (long exposure): exposure around 20,000,000 us. gain around 50

In [1]:
from asi_camera import ASICamera
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import threading

app = FastAPI()

camera: ASICamera  = None

def run():
    uvicorn.run(app, host="0.0.0.0", port=8002)
      
def start_api():
    _api_thread = threading.Thread(target=run)
    _api_thread.start()
    
def create_stream():
    global camera
    camera = ASICamera({ 
        'Camera ID': 1, 
        'Exposure_Time_In_us': 20000, 
        'Gain': 2, 
        'WhiteBalance_Red': 55, 
        'WhiteBalance_Blue': 80,
        'Height': 1080,
        'Width': 1920,
        'Mode': 'RGB24'
    })  
    while True:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + bytearray(camera.get_frame()) + b'\r\n\r\n')
      
@app.get("/")
def get_video_stream():
    return StreamingResponse(create_stream(), media_type='multipart/x-mixed-replace; boundary=frame')

start_api()

INFO:     Started server process [12910]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)


In [2]:
%%HTML 
<img src="http://192.168.0.190:8002/" width=960 />

INFO:     192.168.0.13:45104 - "GET / HTTP/1.1" 200 OK


In [ ]:
print(camera._video.setControlValueManual("ASI_WB_R",55))
print(camera._video.setControlValueManual("ASI_WB_B",88))
